# Introduction to Structural Estimation

<font size="3"> ... background material available at <a href="https://github.com/softecon/talks">https://github.com/softecon/talks</a>  </font>

## Usefulness of Structural Econometric Models

* ***Verification*** How accurately does the computational model solve the underlying equations of the model for the quantities of interest?

* ***Validation*** How accurately does the model represent the reality for the quantities of interest?

* ***Uncertainty Quantification*** How do the various sources of error and uncertainty feed into uncertainty in the model-based prediction of the quantities of interest?

<img src='images/venn_diagram.png' width='750' height='750'>

<center> <b><font size='6'> Maximum-Likelihood Estimation, S = 100</font></b></center>
<img src='images/criterion_maximum_likelihood_100.png' width='750' height='750'>

<center> <b><font size='6'> Method of Moments Estimation, S = 100</font></b></center>
<img src='images/criterion_method_moments_100.png' width='750' height='750'>

<center> <b><font size='6'> Maximum-Likelihood Estimation, S = 1,000</font></b></center>
<img src='images/criterion_maximum_likelihood_1000.png' width='750' height='750'>

## Research Examples

* Eisenhauer, P., Heckman, J., and Mosso, S. (2015).  The Estimation of Dynamic Discrete Choice Models by Maximum Likelihood and the Simulated Method of Moments. *International Economic Review*, 56(2):331-357.

    * Choice of Weighing Matrix
    
    * Choices of Optimization Algorithm

<center> <b><font size='6'> Choice of Weighing Matrix</font></b></center>
<img src='images/weighing_matrix.png' width='750' height='750'>

<center> <b><font size='6'> Choice of Algorithm</font></b></center>
<img src='images/algorithm.png' width='750' height='750'>

* Eisenhauer, P. (2016). The Approximate Solution of Finite-Horizon Discrete Choice Dynamic Programming Models: Revisiting Keane & Wolpin (1994),  *submitted*.



**Interpolation Function**
    
$$\begin{align*}
E \max - \max E = \pi_0 + \sum^4_{j = 1} \pi_{1j} (\max E - \bar{V}_j) +
\sum^4_{j = 1} \pi_{2j} \left(\max E - \bar{V}_j\right)^{\tfrac{1}{2}}
\end{align*}$$

<center> <b><font size='6'>Approximation Scheme and Simulation</font></b></center>
<img src='images/table_approximation_simulation.png' width='750' height='750'>

<center> <b><font size='6'>Approximation Scheme and Estimation</font></b></center>
<img src='images/table_approximation_estimation.png' width='750' height='750'>

## Practical Estimation Exercise

<img src='images/documentation.png' width='750' height='750'>

<center> <b><font size='6'> Curse of Dimensionality</font></b></center>
<img src='images/decision_nodes.png' width='750' height='750'>

**Typical Workflow at Terminal**

* **respy-compare**, compare the observed and simulated economies

* **respy-estimate**, start an estimation run

* **respy-update**, update the initialization files with estimation results

* **respy-modify**, modify the parametrization in the initialization file